<a href="https://colab.research.google.com/github/yogapriya-28/AAIDC-Module-1-Project-RAG-Based-Conversational-Assistant/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -q langchain langchain-openai langchain-community faiss-cpu tiktoken python-dotenv


In [56]:
import os, json
from google.colab import files
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [57]:
from google.colab import userdata
openai.api_key = userdata.get("OPENAI_API_KEY")


In [58]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

with open(file_name, "r", encoding="utf-8") as f:
    publications = json.load(f)

print(f"Loaded {len(publications)} publications.")
print("Example title:", publications[0]['title'])


Saving project_1_publications.json to project_1_publications.json
Loaded 35 publications.
Example title: How to Add Memory to RAG Applications and AI Agents


In [59]:
# Combine title + description for each publication
docs = []
for pub in publications:
    content = f"Title: {pub['title']}\n\nContent: {pub['publication_description']}"
    docs.append(content)

# Split long text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
texts = []
for doc in docs:
    texts.extend(splitter.split_text(doc))

print(f"Total text chunks created: {len(texts)}")


Total text chunks created: 1182


In [60]:
# Use OpenAI’s new embedding model (best 2025 choice)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Build FAISS index
faiss_store = FAISS.from_texts(texts, embeddings)

# Retriever setup
retriever = faiss_store.as_retriever(search_kwargs={"k": 3})
print(" FAISS vector store ready!")


 FAISS vector store ready!


In [61]:
prompt_template = """
You are an AI assistant trained on Ready Tensor publications.
Use the following context to answer the user's question clearly and accurately.

Context:
{context}

Question:
{question}

Answer:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


In [62]:
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

  qa_chain = RetrievalQA.from_chain_type(
      llm=llm,
      retriever=retriever,
      return_source_documents=True,
      chain_type_kwargs={"prompt": prompt}
  )

  print(" RAG Assistant is ready to answer your questions!")


 RAG Assistant is ready to answer your questions!


In [63]:
query = "How can I add memory to a RAG application?"
result = qa_chain.invoke({"query": query})

print(" Question:", query)
print("\n Answer:\n", result["result"])

print("\n Sources:")
for doc in result["source_documents"]:
    print("-", doc.metadata.get("source", "Ready Tensor dataset"))


 Question: How can I add memory to a RAG application?

 Answer:
 To add memory to a RAG application or AI agent, you need to give the application a brain by including the following components:

1. A database for storing user's questions, AI's answers, chat IDs, user's email, etc.
2. A function that retrieves users' previous questions whenever a new question is asked.

By implementing these components, the RAG application will be able to make inferences from previous questions and provide new answers based on both new and previous interactions. This will enhance the application's ability to remember and learn from past interactions, improving its overall performance and user experience.

 Sources:
- Ready Tensor dataset
- Ready Tensor dataset
- Ready Tensor dataset


In [64]:
qa_chain.invoke({"query": "What are best practices for open-source AI repositories?"})


{'query': 'What are best practices for open-source AI repositories?',
 'result': 'Some best practices for open-source AI repositories include:\n1. Providing thorough documentation that explains what the project does, why it matters, how to use it, and whether it can be trusted.\n2. Structuring the repository in a clear and organized manner to make it easy for others to navigate and understand.\n3. Managing environment and dependencies effectively to ensure reproducibility.\n4. Considering licensing and legal considerations to protect intellectual property and ensure proper usage.\n5. Maintaining high code quality standards to make the project more accessible and usable for the community. \n\nBy following these best practices, AI practitioners can create repositories that are valuable resources for the wider community and contribute to the advancement of open science principles in the AI landscape.',
 'source_documents': [Document(id='ac83e7e8-e0ec-4146-87bd-b96ef183aa80', metadata={}, 

In [65]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# -------------------------------
# Chat Session Setup
# -------------------------------
chat_history = []

chat_output = widgets.Output(
    layout={
        'border': '1px solid #444',
        'height': '400px',
        'overflow_y': 'auto',
        'padding': '10px',
        'background': '#f9f9f9'
    }
)

chat_input = widgets.Text(
    placeholder='Type your question here...',
    description='You:',
    layout=widgets.Layout(width='80%')
)

send_button = widgets.Button(
    description="Send",
    button_style='success',
    layout=widgets.Layout(width='100px')
)

clear_button = widgets.Button(
    description="Clear",
    button_style='danger',
    layout=widgets.Layout(width='100px')
)

display(widgets.VBox([
    chat_output,
    widgets.HBox([chat_input, send_button, clear_button])
]))

# -------------------------------
# Update Chat Display
# -------------------------------
def update_chat_display():
    """Redraw all messages"""
    with chat_output:
        clear_output(wait=True)
        for user_msg, assistant_msg in chat_history:
            display(HTML(f"""
            <div style="background-color:#E8F0FF; padding:8px; border-radius:10px; margin:5px 0;">
                <b>You:</b> {user_msg}
            </div>
            <div style="background-color:#E6FFE6; padding:8px; border-radius:10px; margin:5px 0;">
                <b>Assistant:</b> {assistant_msg}
            </div>
            """))

# -------------------------------
# Send Message Handler
# -------------------------------
def send_message(_):
    query = chat_input.value.strip()
    if not query:
        return

    chat_input.value = ""
    with chat_output:
        display(HTML(f"<div style='color:gray'>⏳ Thinking...</div>"))

    try:
        result = qa_chain.invoke({"query": query})
        answer = result.get("result", "⚠️ No answer generated.")
    except Exception as e:
        answer = f"❌ Error: {e}"

    chat_history.append((query, answer))
    update_chat_display()

# -------------------------------
# Clear Chat Handler
# -------------------------------
def clear_chat(_):
    chat_history.clear()
    update_chat_display()

# -------------------------------
# Event Bindings
# -------------------------------
send_button.on_click(send_message)
clear_button.on_click(clear_chat)

def on_enter(change):
    if change['name'] == 'value' and change['new'].endswith('\n'):
        send_message(None)

chat_input.observe(on_enter, names='value')


In [42]:
!pip install streamlit -q
!wget -q -O cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared.deb


(Reading database ... 126722 files and directories currently installed.)
Preparing to unpack cloudflared.deb ...
Unpacking cloudflared (2025.10.0) over (2025.10.0) ...
Setting up cloudflared (2025.10.0) ...
Processing triggers for man-db (2.10.2-1) ...


In [51]:
%%writefile RAG.py
import streamlit as st
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import json, os, time

# ------------------------------
# PAGE CONFIGURATION
# ------------------------------
st.set_page_config(page_title="🧠 Ready Tensor RAG Assistant", page_icon="⚡", layout="wide")

st.markdown("""
<style>
body {background-color: #050510; color: #EAEAEA;}
h1, h4 {color: #8be9fd;}
footer {visibility: hidden;}
div.stTextInput > div > input {
    background-color: #0f0f1f;
    color: #fff;
    border: 1px solid #333;
    border-radius: 10px;
    padding: 10px;
}
div.stButton > button {
    background: linear-gradient(90deg, #7b2ff7 0%, #f107a3 100%);
    color: white; font-weight: bold; border-radius: 8px;
}
.chatbox-user {
    background: linear-gradient(90deg, #518aed 0%, #272752 100%);
    border-radius: 12px; padding: 10px; margin: 8px 0;
}
.chatbox-bot {
    background: linear-gradient(90deg, #43107a 0%, #f107a3 100%);
    border-radius: 12px; padding: 10px; margin: 8px 0;
    color: #fff;
    animation: fadeIn 0.8s ease;
}
@keyframes fadeIn { from {opacity: 0;} to {opacity: 1;} }
</style>
""", unsafe_allow_html=True)

# ------------------------------
# HEADER
# ------------------------------
st.title("🧠 RAG-Based Conversational Assistant for Ready Tensor Publications")
st.markdown("<h4>AAIDC Project 1 | Built by <b>@yogapriya</b></h4>", unsafe_allow_html=True)
st.markdown("<hr>", unsafe_allow_html=True)

# ------------------------------
# LOAD AND PROCESS DATA
# ------------------------------
@st.cache_resource
def load_data():
    with open("/content/project_1_publications.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    texts = [f"{item['title']} — {item.get('publication_description', '')}" for item in data]
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = splitter.create_documents(texts)
    return docs

docs = load_data()

# ------------------------------
# VECTOR STORE AND RETRIEVAL
# ------------------------------
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="You are a helpful AI assistant trained on Ready Tensor publications.\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2),
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template}
)

# ------------------------------
# CHAT INTERFACE
# ------------------------------
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

st.markdown("### 💬 Ask your question:")
user_input = st.text_input("Type your message here:")

col1, col2 = st.columns(2)
with col1:
    send = st.button("🚀 Send", use_container_width=True)
with col2:
    clear = st.button("🧹 Clear Chat", use_container_width=True)

# Clear Chat
if clear:
    st.session_state.chat_history = []
    st.success("✨ Chat history cleared!")

# Send Message
if send and user_input:
    with st.spinner("Thinking... 🤖"):
        result = qa_chain.invoke({"question": user_input})
        answer = result.get("answer", result.get("result", "No response generated."))
        st.session_state.chat_history.append((user_input, answer))

# ------------------------------
# DISPLAY CHAT HISTORY
# ------------------------------
st.markdown("### 🧾 Conversation History")

chat_container = st.container()
for user_msg, bot_msg in st.session_state.chat_history:
    chat_container.markdown(f"<div class='chatbox-user'><b>You:</b> {user_msg}</div>", unsafe_allow_html=True)
    with chat_container:
        placeholder = st.empty()
        displayed_text = ""
        for char in bot_msg:
            displayed_text += char
            placeholder.markdown(f"<div class='chatbox-bot'><b>Assistant:</b> {displayed_text}</div>", unsafe_allow_html=True)
            time.sleep(0.01)

st.markdown("<hr><p style='text-align:center; color:#777;'>⚙️ Ready Tensor | AAIDC Module 1 | RAG Assistant Demo (2025)</p>", unsafe_allow_html=True)


Overwriting RAG.py


In [44]:
import threading, time, os, re

# ✅ Step 1: Start Streamlit app in background
def run_streamlit():
    os.system("streamlit run/content/RAG.py --server.port 8501 > /content/streamlit.log 2>&1")

thread = threading.Thread(target=run_streamlit)
thread.start()

time.sleep(5)

# ✅ Step 2: Start Cloudflare Tunnel in background
os.system('cloudflared tunnel --url http://localhost:8501 --no-autoupdate > /content/cf.log 2>&1 &')

time.sleep(8)

# ✅ Step 3: Extract the public URL from Cloudflare logs
with open("/content/cf.log", "r") as f:
    log_data = f.read()

url_match = re.search(r"https://[-0-9a-z]+\.trycloudflare\.com", log_data)
if url_match:
    public_url = url_match.group(0)
    print("✅ Your Streamlit app is live! Click below:\n")
    print(public_url)
else:
    print("⚠️ Could not detect public URL yet. Try re-running this cell after 10 seconds.")


✅ Your Streamlit app is live! Click below:

https://reverse-promotes-array-previously.trycloudflare.com
